# Human-in-the-Loop Workflow z Microsoft Agent Framework

## 🎯 Cele nauki

W tym notatniku nauczysz się, jak wdrożyć przepływy pracy **human-in-the-loop** za pomocą `RequestInfoExecutor` w Microsoft Agent Framework. Ten potężny wzorzec pozwala na wstrzymanie przepływów pracy AI w celu uzyskania danych od człowieka, dzięki czemu agenci stają się interaktywni, a ludzie mają kontrolę nad kluczowymi decyzjami.

## 🔄 Co to jest Human-in-the-Loop?

**Human-in-the-loop (HITL)** to wzorzec projektowy, w którym agenci AI wstrzymują wykonanie, aby poprosić o dane od człowieka przed kontynuowaniem. Jest to kluczowe w przypadku:

- ✅ **Decyzji krytycznych** - Uzyskanie zgody człowieka przed podjęciem ważnych działań
- ✅ **Niejasnych sytuacji** - Pozwolenie ludziom na wyjaśnienie, gdy AI jest niepewne
- ✅ **Preferencji użytkownika** - Pytanie użytkowników o wybór między różnymi opcjami
- ✅ **Zgodności i bezpieczeństwa** - Zapewnienie nadzoru człowieka w operacjach regulowanych
- ✅ **Interaktywnych doświadczeń** - Tworzenie agentów konwersacyjnych, którzy reagują na dane od użytkownika

## 🏗️ Jak to działa w Microsoft Agent Framework

Framework oferuje trzy kluczowe komponenty dla HITL:

1. **`RequestInfoExecutor`** - Specjalny executor, który wstrzymuje przepływ pracy i emituje `RequestInfoEvent`
2. **`RequestInfoMessage`** - Klasa bazowa dla typowych danych żądania wysyłanych do ludzi
3. **`RequestResponse`** - Koreluje odpowiedzi człowieka z pierwotnymi żądaniami za pomocą `request_id`

**Wzorzec przepływu pracy:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Nasz przykład: Rezerwacja hotelu z potwierdzeniem użytkownika

Rozbudujemy warunkowy przepływ pracy, dodając potwierdzenie człowieka **przed** zasugerowaniem alternatywnych destynacji:

1. Użytkownik prosi o destynację (np. "Paryż")
2. `availability_agent` sprawdza dostępność pokoi
3. **Jeśli brak pokoi** → `confirmation_agent` pyta "Czy chcesz zobaczyć alternatywy?"
4. Przepływ pracy **wstrzymuje się** za pomocą `RequestInfoExecutor`
5. **Człowiek odpowiada** "tak" lub "nie" przez konsolę
6. `decision_manager` kieruje na podstawie odpowiedzi:
   - **Tak** → Pokazuje alternatywne destynacje
   - **Nie** → Anuluje żądanie rezerwacji
7. Wyświetlenie ostatecznego wyniku

To pokazuje, jak dać użytkownikom kontrolę nad sugestiami agenta!

---

Zaczynajmy! 🚀


## Krok 1: Importowanie wymaganych bibliotek

Importujemy standardowe komponenty Agent Framework oraz **klasy specyficzne dla interakcji z człowiekiem**:
- `RequestInfoExecutor` - Egzekutor, który zatrzymuje przepływ pracy w celu uzyskania danych od człowieka
- `RequestInfoEvent` - Zdarzenie emitowane, gdy wymagane są dane od człowieka
- `RequestInfoMessage` - Klasa bazowa dla typowanych ładunków żądań
- `RequestResponse` - Koreluje odpowiedzi człowieka z żądaniami
- `WorkflowOutputEvent` - Zdarzenie wykrywające wyniki przepływu pracy


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Krok 2: Zdefiniuj modele Pydantic dla uporządkowanych wyników

Te modele definiują **schemat**, który agenci będą zwracać. Zachowujemy wszystkie modele z warunkowego przepływu pracy i dodajemy:

**Nowość dla Human-in-the-Loop:**
- `HumanFeedbackRequest` - Podklasa `RequestInfoMessage`, która definiuje dane żądania wysyłane do ludzi
  - Zawiera `prompt` (pytanie do zadania) i `destination` (kontekst dotyczący niedostępnego miasta)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Krok 3: Utwórz narzędzie do rezerwacji hotelu

To samo narzędzie z warunkowego przepływu pracy - sprawdza dostępność pokoi w miejscu docelowym.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Krok 4: Zdefiniuj funkcje warunkowe dla routingu

Potrzebujemy **czterech funkcji warunkowych** dla naszego procesu z udziałem człowieka:

**Z warunkowego workflow:**
1. `has_availability_condition` - Kieruje, gdy hotele SĄ dostępne
2. `no_availability_condition` - Kieruje, gdy hotele NIE są dostępne

**Nowe dla procesu z udziałem człowieka:**
3. `user_wants_alternatives_condition` - Kieruje, gdy użytkownik odpowiada "tak" na alternatywy
4. `user_declines_alternatives_condition` - Kieruje, gdy użytkownik odpowiada "nie" na alternatywy


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Krok 5: Utwórz Wykonawcę Decision Manager

To jest **sedno wzorca human-in-the-loop**! `DecisionManager` to niestandardowy `Executor`, który:

1. **Otrzymuje opinię od człowieka** za pośrednictwem obiektów `RequestResponse`
2. **Przetwarza decyzję użytkownika** (tak/nie)
3. **Kieruje przepływem pracy**, wysyłając wiadomości do odpowiednich agentów

Kluczowe cechy:
- Używa dekoratora `@handler`, aby udostępniać metody jako kroki przepływu pracy
- Otrzymuje `RequestResponse[HumanFeedbackRequest, str]`, zawierający zarówno oryginalne żądanie, jak i odpowiedź użytkownika
- Generuje proste wiadomości "tak" lub "nie", które uruchamiają nasze funkcje warunkowe


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Krok 6: Utwórz niestandardowy wykonawca wyświetlania

Ten sam wykonawca wyświetlania z warunkowego przepływu pracy - generuje ostateczne wyniki jako wynik przepływu pracy.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Krok 7: Załaduj zmienne środowiskowe

Skonfiguruj klienta LLM (GitHub Models, Azure OpenAI lub OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Krok 8: Tworzenie agentów AI i wykonawców

Tworzymy **sześć komponentów przepływu pracy**:

**Agenci (opakowani w AgentExecutor):**
1. **availability_agent** - Sprawdza dostępność hoteli za pomocą narzędzia
2. **confirmation_agent** - 🆕 Przygotowuje prośbę o potwierdzenie od człowieka
3. **alternative_agent** - Proponuje alternatywne miasta (gdy użytkownik odpowiada "tak")
4. **booking_agent** - Zachęca do rezerwacji (gdy pokoje są dostępne)
5. **cancellation_agent** - 🆕 Obsługuje wiadomość o anulowaniu (gdy użytkownik odpowiada "nie")

**Specjalni wykonawcy:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, który wstrzymuje przepływ pracy w oczekiwaniu na dane od człowieka
7. **decision_manager** - 🆕 Niestandardowy wykonawca, który kieruje na podstawie odpowiedzi człowieka (już zdefiniowany powyżej)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Krok 9: Budowa przepływu pracy z udziałem człowieka

Teraz tworzymy wykres przepływu pracy z **warunkowym kierowaniem**, uwzględniając ścieżkę z udziałem człowieka:

**Struktura przepływu pracy:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Kluczowe połączenia:**
- `availability_agent → confirmation_agent` (gdy brak wolnych pokoi)
- `confirmation_agent → prepare_human_request` (zmiana typu)
- `prepare_human_request → request_info_executor` (pauza dla człowieka)
- `request_info_executor → decision_manager` (zawsze - dostarcza RequestResponse)
- `decision_manager → alternative_agent` (gdy użytkownik mówi "tak")
- `decision_manager → cancellation_agent` (gdy użytkownik mówi "nie")
- `availability_agent → booking_agent` (gdy pokoje są dostępne)
- Wszystkie ścieżki kończą się na `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Krok 10: Uruchom Test Case 1 - Miasto BEZ Dostępności (Paryż z Potwierdzeniem Ludzkim)

Ten test pokazuje **pełny cykl z udziałem człowieka**:

1. Żądanie hotelu w Paryżu
2. availability_agent sprawdza → Brak pokoi
3. confirmation_agent tworzy pytanie skierowane do człowieka
4. request_info_executor **wstrzymuje przepływ pracy** i emituje `RequestInfoEvent`
5. **Aplikacja wykrywa zdarzenie i wyświetla pytanie użytkownikowi w konsoli**
6. Użytkownik wpisuje "tak" lub "nie"
7. Aplikacja wysyła odpowiedź za pomocą `send_responses_streaming()`
8. decision_manager kieruje na podstawie odpowiedzi
9. Wyświetlany jest ostateczny wynik

**Kluczowy schemat:**
- Użyj `workflow.run_stream()` dla pierwszej iteracji
- Użyj `workflow.send_responses_streaming(pending_responses)` dla kolejnych iteracji
- Nasłuchuj `RequestInfoEvent`, aby wykryć, kiedy potrzebne są dane od człowieka
- Nasłuchuj `WorkflowOutputEvent`, aby przechwycić ostateczne wyniki


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Krok 11: Uruchom Test Przypadek 2 - Miasto Z Dostępnością (Sztokholm - Bez Wymaganej Interwencji Człowieka)

Ten test pokazuje **bezpośrednią ścieżkę** w przypadku dostępności pokoi:

1. Żądanie hotelu w Sztokholmie  
2. availability_agent sprawdza → Pokoje dostępne ✅  
3. booking_agent sugeruje rezerwację  
4. display_result pokazuje potwierdzenie  
5. **Brak wymaganej interwencji człowieka!**

Przepływ całkowicie omija ścieżkę z udziałem człowieka, gdy pokoje są dostępne.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Kluczowe informacje i najlepsze praktyki w modelu Human-in-the-Loop

### ✅ Czego się nauczyłeś:

#### 1. **Wzorzec RequestInfoExecutor**
Wzorzec human-in-the-loop w Microsoft Agent Framework wykorzystuje trzy kluczowe komponenty:
- `RequestInfoExecutor` - Wstrzymuje przepływ pracy i emituje zdarzenia
- `RequestInfoMessage` - Klasa bazowa dla typowanych danych żądania (rozszerzaj ją!)
- `RequestResponse` - Koreluje odpowiedzi użytkownika z pierwotnymi żądaniami

**Kluczowe zrozumienie:**
- `RequestInfoExecutor` NIE zbiera danych wejściowych samodzielnie - jedynie wstrzymuje przepływ pracy
- Kod aplikacji musi nasłuchiwać `RequestInfoEvent` i zbierać dane wejściowe
- Musisz wywołać `send_responses_streaming()` z mapowaniem `request_id` na odpowiedź użytkownika

#### 2. **Wzorzec wykonania strumieniowego**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Architektura oparta na zdarzeniach**
Nasłuchuj określonych zdarzeń, aby kontrolować przepływ pracy:
- `RequestInfoEvent` - Wymagane dane od człowieka (przepływ pracy wstrzymany)
- `WorkflowOutputEvent` - Dostępny jest ostateczny wynik (przepływ pracy zakończony)
- `WorkflowStatusEvent` - Zmiany stanu (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, itd.)

#### 4. **Niestandardowe wykonawcy z @handler**
`DecisionManager` pokazuje, jak tworzyć wykonawców, którzy:
- Używają dekoratora `@handler`, aby udostępniać metody jako kroki przepływu pracy
- Otrzymują typowane wiadomości (np. `RequestResponse[HumanFeedbackRequest, str]`)
- Kierują przepływem pracy, wysyłając wiadomości do innych wykonawców
- Uzyskują dostęp do kontekstu za pomocą `WorkflowContext`

#### 5. **Warunkowe kierowanie decyzjami człowieka**
Możesz tworzyć funkcje warunkowe oceniające odpowiedzi człowieka:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Zastosowania w rzeczywistości:

1. **Przepływy zatwierdzania**
   - Uzyskiwanie zgody menedżera przed przetworzeniem raportów wydatków
   - Wymaganie przeglądu przez człowieka przed wysłaniem automatycznych e-maili
   - Potwierdzanie transakcji o wysokiej wartości przed ich realizacją

2. **Moderacja treści**
   - Oznaczanie wątpliwych treści do przeglądu przez człowieka
   - Prośba do moderatorów o podjęcie ostatecznej decyzji w trudnych przypadkach
   - Eskalacja do człowieka, gdy pewność AI jest niska

3. **Obsługa klienta**
   - Pozwolenie AI na automatyczne obsługiwanie rutynowych pytań
   - Eskalacja złożonych problemów do ludzkich agentów
   - Pytanie klienta, czy chce porozmawiać z człowiekiem

4. **Przetwarzanie danych**
   - Prośba do ludzi o rozwiązanie niejednoznacznych wpisów danych
   - Potwierdzanie interpretacji AI w przypadku niejasnych dokumentów
   - Pozwalanie użytkownikom na wybór między wieloma poprawnymi interpretacjami

5. **Systemy krytyczne dla bezpieczeństwa**
   - Wymaganie potwierdzenia przez człowieka przed nieodwracalnymi działaniami
   - Uzyskiwanie zgody przed dostępem do wrażliwych danych
   - Potwierdzanie decyzji w regulowanych branżach (opieka zdrowotna, finanse)

6. **Interaktywne agenty**
   - Tworzenie botów konwersacyjnych, które zadają pytania uzupełniające
   - Projektowanie kreatorów prowadzących użytkowników przez złożone procesy
   - Budowanie agentów współpracujących z ludźmi krok po kroku

### 🔄 Porównanie: Z vs Bez Human-in-the-Loop

| Funkcja | Warunkowy przepływ pracy | Przepływ pracy Human-in-the-Loop |
|---------|--------------------------|----------------------------------|
| **Wykonanie** | Pojedyncze `workflow.run()` | Pętla z `run_stream()` + `send_responses_streaming()` |
| **Dane wejściowe użytkownika** | Brak (w pełni zautomatyzowane) | Interaktywne zapytania przez `input()` lub UI |
| **Komponenty** | Agenci + Wykonawcy | + RequestInfoExecutor + DecisionManager |
| **Zdarzenia** | Tylko AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, itd. |
| **Wstrzymywanie** | Brak wstrzymywania | Przepływ pracy wstrzymany w RequestInfoExecutor |
| **Kontrola człowieka** | Brak kontroli człowieka | Ludzie podejmują kluczowe decyzje |
| **Zastosowanie** | Automatyzacja | Współpraca i nadzór |

### 🚀 Zaawansowane wzorce:

#### Wielokrotne punkty decyzji człowieka
Możesz mieć wiele węzłów `RequestInfoExecutor` w tym samym przepływie pracy:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Obsługa limitów czasu
Zaimplementuj limity czasu dla odpowiedzi człowieka:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integracja z zaawansowanym interfejsem użytkownika
Zamiast `input()`, zintegrować z interfejsem webowym, Slackiem, Teams, itd.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Warunkowy Human-in-the-Loop
Proś o dane od człowieka tylko w określonych sytuacjach:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Najlepsze praktyki:

1. **Zawsze rozszerzaj RequestInfoMessage**
   - Zapewnia bezpieczeństwo typów i walidację
   - Umożliwia bogaty kontekst dla renderowania UI
   - Wyjaśnia cel każdego typu żądania

2. **Używaj opisowych zapytań**
   - Uwzględnij kontekst tego, o co prosisz
   - Wyjaśnij konsekwencje każdego wyboru
   - Zachowaj pytania proste i jasne

3. **Obsługuj nieoczekiwane dane wejściowe**
   - Waliduj odpowiedzi użytkownika
   - Zapewnij domyślne wartości dla nieprawidłowych danych
   - Podawaj jasne komunikaty o błędach

4. **Śledź identyfikatory żądań**
   - Używaj korelacji między request_id a odpowiedziami
   - Nie próbuj zarządzać stanem ręcznie

5. **Projektuj dla nieblokowania**
   - Nie blokuj wątków czekających na dane wejściowe
   - Używaj wzorców asynchronicznych
   - Obsługuj równoczesne instancje przepływu pracy

### 📚 Powiązane koncepcje:

- **Agent Middleware** - Przechwytywanie wywołań agentów (poprzedni notebook)
- **Zarządzanie stanem przepływu pracy** - Utrzymywanie stanu przepływu pracy między uruchomieniami
- **Współpraca wieloagentowa** - Łączenie human-in-the-loop z zespołami agentów
- **Architektury oparte na zdarzeniach** - Budowanie reaktywnych systemów opartych na zdarzeniach

---

### 🎓 Gratulacje!

Opanowałeś przepływy pracy human-in-the-loop w Microsoft Agent Framework! Teraz wiesz, jak:
- ✅ Wstrzymywać przepływy pracy, aby zbierać dane od człowieka
- ✅ Używać RequestInfoExecutor i RequestInfoMessage
- ✅ Obsługiwać wykonanie strumieniowe za pomocą zdarzeń
- ✅ Tworzyć niestandardowych wykonawców z @handler
- ✅ Kierować przepływem pracy na podstawie decyzji człowieka
- ✅ Budować interaktywne agenty AI współpracujące z ludźmi

**To kluczowy wzorzec do budowania wiarygodnych, kontrolowalnych systemów AI!** 🚀



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za autorytatywne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
